# Simple pendulum

In [1]:
import numpy as np
from scipy.integrate import odeint
import pandas as pd
import warnings
pd.set_option('display.float_format', '{:0.8f}'.format)
import operator

Upright Pendulum link

https://www.12000.org/my_notes/cart_motion/report.htm

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

# Function to compute derivatives
def pendulum_rhs(t, y, gamma, L=1):
    """
    Function to compute derivatives for simple pendulum with damping
    
    Parameters:
        t : float
            Time
        y : array_like
            Vector containing [theta, omega], where
            theta is the angle and omega is the angular velocity
        gamma : float
            Damping coefficient
        L : float
            Length of the pendulum
        
    Returns:
        dydt : array_like
            Vector containing [omega, alpha], where
            omega is the angular velocity and alpha is the angular acceleration
    """
    theta, omega = y
    alpha = - (9.81 / L) * np.sin(theta) - gamma * omega
    return [omega, alpha]

# Parameters
theta0 = np.pi / 4  # Initial angle (radians)
omega0 = 0.0        # Initial angular velocity (radians per second)
gamma = 0.0       # Damping coefficient
L = 1.0             # Length of the pendulum (meters)
t_span = (0, 10)    # Time span for the simulation

# Function to integrate the system of ODEs
def integrate_pendulum(t_span, y0, gamma, L):

    sol = solve_ivp(lambda t, y: pendulum_rhs(t, y, gamma, L), t_span, y0, method='RK45', t_eval=np.linspace(*t_span, 1000))
    return sol

# Integrate the pendulum system
sol = integrate_pendulum(t_span, [theta0, omega0], gamma, L)

# Plot the results
plt.figure(figsize=(10, 5))
plt.plot(sol.t, sol.y[0], label='Angle (radians)')
plt.plot(sol.t, sol.y[1], label='Angular velocity (rad/s)')
plt.title('Damped Simple Pendulum Simulation using scipy.solve_ivp')
plt.xlabel('Time (s)')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

In [103]:
IC_df = pd.read_csv("parameters/init_cond_simp_pend.csv")

In [104]:
# IC_df = IC_df[0:2]
IC_df

In [105]:
# Mechanical eEnergy level
0.5*(IC_df["omega"])**2 + 9.81*(1-np.cos(IC_df["theta"]))

In [106]:
params_df = pd.read_csv("parameters/pend_param.csv")
params_df


In [107]:
g = 9.81   # Acceleration due to gravity (m/s^2)


### Synthesizing data from different ICs

### Synthesizing data from different ICs

In [116]:
L = 5.0
# y_shift = 0.9 * L
# y_shift = 0

# Time span
t_span = (0.0, 10)  # from 0 to 10 seconds
#Valuation points
t_eval_ = np.linspace(t_span[0], t_span[1], 100)
data_matrix_df_list = []


for param_index in params_df.index:
    params = params_df.loc[param_index]
    # Define parameters
    m_c = params['m_c']  # Mass of the cart (kg)
    m_p = params['m_p']  # Mass of the pendulum (kg)
    l = params['l']    # Length of the pendulum (m)
    for IC_index in IC_df.index:
        IC = IC_df.loc[IC_index]
        y0 = IC.values
                # Parameters
        theta0 = IC["theta"]  # Initial angle (radians)
        omega0 = IC["omega"]        # Initial angular velocity (radians per second)
        gamma = 0.0         # Damping coefficient
        # Solve the ODEs
        sol = solve_ivp(lambda t, y: pendulum_rhs(t, y, gamma, L), t_span, [theta0, omega0], method='RK45', t_eval=t_eval_)
        sol_df = pd.DataFrame(sol.y.T, columns=["theta", "omega"])
        sol_df["x"] = L*np.sin(sol_df["theta"])
        sol_df["y"] = -L*np.cos(sol_df["theta"])
        data_matrix_df_list.append(sol_df[["x", "y"]])
        # if IC_index == 0:
        #     # Plot the results
        #     plt.figure(figsize=(10, 6))
        #     plt.plot(sol.t, sol.y[0], label='Cart Position (x)')
        #     plt.plot(sol.t, sol.y[2], label='Pendulum Angle (theta)')
        #     plt.xlabel('Time (s)')
        #     plt.ylabel('Position (m) / Angle (rad)')
        #     plt.title('Upright Pendulum on Moving Cart')
        #     plt.legend()
        #     plt.grid(True)
        #     plt.show()

data_matrix_df = pd.concat(data_matrix_df_list, ignore_index=True)
data_matrix_df

In [117]:
(3/4)*np.pi

In [110]:
data_matrix_df[["x","y"]].plot()

In [87]:
data_matrix_df_appended["d(theta) /dt"]

In [43]:
from copy import deepcopy
new_df = deepcopy(data_matrix_df_appended)

new_df["energy"] = 0.5*((new_df["d(x) /dt"])**2 + (new_df["d(y) /dt"])**2) +  9.81*new_df["y"]

In [44]:
new_df.plot()

### Calculating the derivatives of data

In [118]:
from dae_finder import der_matrix_calculator

In [119]:
delta_t = t_eval_[1]- t_eval_[0]
data_matrix_features = data_matrix_df_list[0].columns
for ind, data_matrix_ in enumerate(data_matrix_df_list):
    derr_matrix = der_matrix_calculator(data_matrix_, delta_t)
    data_matrix_df_list[ind] = pd.concat([data_matrix_.iloc[:-1], derr_matrix], axis=1)

data_matrix_df_appended = pd.concat(data_matrix_df_list, ignore_index=True)
data_matrix_df_appended

In [30]:
delta_t = t_eval_[1]- t_eval_[0]
data_matrix_features = data_matrix_df_list[0].columns
for ind, data_matrix_ in enumerate(data_matrix_df_list):
    derr_matrix = der_matrix_calculator(data_matrix_, delta_t)
    data_matrix_df_list[ind] = pd.concat([data_matrix_.iloc[:-1], derr_matrix], axis=1)

data_matrix_df_appended = pd.concat(data_matrix_df_list, ignore_index=True)
data_matrix_df_appended

In [ ]:
data_matrix_df_appended["x"]

In [122]:
# data_matrix_df = data_matrix_df_appended[["x","y"]]
# data_matrix_df = pd.concat([data_matrix_df, data_matrix_df_appended[["d(u) /dt"]]], axis=1)
data_matrix_df = data_matrix_df_appended[["x","y", "d(x) /dt", "d(y) /dt"]]
data_matrix_df

## Forming candiate library

In [113]:
from sklearn.preprocessing import FunctionTransformer
from copy import deepcopy

def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))


def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))


data_matrix_df_with_trig = deepcopy(data_matrix_df)
data_matrix_df_with_trig["sin(theta)"] = sin_transformer(1).fit_transform(data_matrix_df_with_trig)["theta"]
data_matrix_df_with_trig["cos(theta)"] = cos_transformer(1).fit_transform(data_matrix_df_with_trig)["theta"]

In [123]:
from dae_finder import PolyFeatureMatrix

poly_feature_ob = PolyFeatureMatrix(2)

candidate_lib_full = poly_feature_ob.fit_transform(data_matrix_df)

In [124]:
candidate_lib_full = candidate_lib_full.drop(["1"], axis=1)
candidate_lib_full

In [319]:
candid_lib_comb = pd.concat([candidate_lib_full, data_matrix_df_with_trig[["cos(theta)", "sin(theta)"]]], axis=1)
candid_lib_comb

### SVD analysis

In [136]:
from sklearn import decomposition
pca_1 = decomposition.PCA()
pca_1.fit(candidate_lib_full)

# pca_2 = decomposition.PCA()
# pca_2.fit(mean_candidate_lib)

# pca_3 = decomposition.PCA()
# pca_3.fit(selected_data_matrix_df)

pca_2 = decomposition.PCA()
pca_2.fit(candidate_lib_full.drop(["x^2", "x d(x) /dt"],axis=1))

pca_3 = decomposition.PCA()
pca_3.fit(candidate_lib_full.drop(["x^2", "x d(x) /dt", "y"],axis=1))


# singular_values = pca_1.singular_values_
# mean_singular_values = pca_2.singular_values_

var_expl_ratio = pca_1.explained_variance_ratio_
theta_dot_sq_rem_expl_ratio = pca_2.explained_variance_ratio_
theta_dot_rem_expl_ratio = pca_3.explained_variance_ratio_
# data_var_expl_ratio_E = pca_4.explained_variance_

# var_expl_ratio_E_rem = pca_5.explained_variance_


In [137]:
from matplotlib import pyplot as plt

plt.scatter(np.arange(len(var_expl_ratio)),np.log(var_expl_ratio))
plt.grid()
# for x, y in zip(np.arange(len(candid_lib_sing_values)),np.log(candid_lib_sing_values)):
#     plt.text(x,y,y)

In [138]:
from matplotlib import pyplot as plt

plt.scatter(np.arange(len(theta_dot_sq_rem_expl_ratio)),np.log(theta_dot_sq_rem_expl_ratio))
plt.grid()
# for x, y in zip(np.arange(len(candid_lib_sing_values)),np.log(candid_lib_sing_values)):
#     plt.text(x,y,y)

In [139]:
from matplotlib import pyplot as plt

plt.scatter(np.arange(len(theta_dot_rem_expl_ratio)),np.log(theta_dot_rem_expl_ratio))
plt.grid()
# for x, y in zip(np.arange(len(candid_lib_sing_values)),np.log(candid_lib_sing_values)):
#     plt.text(x,y,y)

### Finding the remaining algebraic relationships

In [125]:
from dae_finder import AlgModelFinder





from dae_finder import sequentialThLin, AlgModelFinder
seq_th_model = sequentialThLin(fit_intercept=True, coef_threshold= 0.1)
algebraic_model_th = AlgModelFinder(custom_model=True, custom_model_ob= seq_th_model)


In [126]:
algebraic_model_th.fit(candidate_lib_full, scale_columns= False)

In [127]:
algebraic_model_th.best_models(5)

In [128]:
seq_th_model = sequentialThLin(fit_intercept=True, coef_threshold= 0.05)
algebraic_model_th = AlgModelFinder(custom_model=True, custom_model_ob= seq_th_model)

algebraic_model_th.fit(candidate_lib_full.drop(["x^2", "x d(x) /dt"], axis=1), scale_columns= False)

In [129]:
algebraic_model_th.best_models(5)

In [98]:
0.99995441 - 0.99993971

In [100]:
candidate_lib_full["d(y) /dt"].plot()

In [328]:
plt.plcandid_lib_comb["theta^2"]

In [130]:
#Use lasso model by default
algebraic_model_1 = AlgModelFinder(model_id='lasso', alpha=0.3, fit_intercept=True)
algebraic_model_1.fit(candidate_lib_full, scale_columns= True)


algebraic_model_1.best_models(5)

In [131]:
#Use lasso model by default
algebraic_model_1 = AlgModelFinder(model_id='lasso', alpha=0.3, fit_intercept=True)
algebraic_model_1.fit(candidate_lib_full.drop(["x^2", "x d(x) /dt"], axis=1), scale_columns= True)


algebraic_model_1.best_models(5)

In [303]:
candidate_lib_full["y^2"]

In [304]:
from sklearn.linear_model import LinearRegression

lin_model = LinearRegression(fit_intercept=True)
lin_model.fit(X=candidate_lib_full[["x^2"]], y=candidate_lib_full["y^2"])


In [305]:
lin_model.coef_
lin_model.intercept_

In [306]:
plt.plot(candidate_lib_full["x^2"], -0.7310178*candidate_lib_full["y^2"] + 0.41957709*-0.7310178*candidate_lib_full["y"])

In [94]:
(3/2)*np.pi

In [89]:
thet = (3/2)*np.pi
theta_dot = 6

0.5*(theta_dot)**2 + 9.81*(1-np.cos(thet))

In [90]:
thet = (3/2)*np.pi
theta_dot = -6

0.5*(theta_dot)**2 + 9.81*(1-np.cos(thet))

In [92]:
thet = -(3/2)*np.pi
theta_dot = 6

0.5*(theta_dot)**2 + 9.81*(1-np.cos(thet))

In [93]:
thet = -(3/2)*np.pi
theta_dot = -6

0.5*(theta_dot)**2 + 9.81*(1-np.cos(thet))

In [82]:
thet = 3/4*np.pi
theta_dot = 0

0.5*(theta_dot)**2 + 9.81*(1-np.cos(thet))

## Testing dynamics finding 

In [327]:
refined_candid_lib = candidate_lib_full.drop(['x^2'], axis=1)
# refined_candid_lib = candidate_lib_full.drop(['1'], axis=1)
# refined_candid_lib = pd.concat([refined_candid_lib, dummy_der[['d(omega) /dt']]], axis=1)

In [328]:
refined_candid_lib

## Refined candidate library is able to find the model 

In [329]:
data_matrix_df_appended["d(d(x) /dt) /dt"]

In [330]:
from dae_finder import sequentialThLin

seq_th_model = sequentialThLin(fit_intercept=False, coef_threshold=0.2)

seq_th_model.fit(X=refined_candid_lib,  y=data_matrix_df_appended["d(d(x) /dt) /dt"])

In [331]:
dict(zip(seq_th_model.feature_names_in_, seq_th_model.coef_))

#### Using lasso

In [332]:
from sklearn.preprocessing import StandardScaler
s_scaler = StandardScaler(with_std=True, with_mean=False)
scaled_cand_lib = pd.DataFrame(s_scaler.fit_transform(refined_candid_lib), columns=s_scaler.feature_names_in_)

In [333]:
from sklearn.linear_model import Lasso
alg_lasso = Lasso(fit_intercept=False, alpha=0.3)
alg_lasso.fit(X=refined_candid_lib,  y=data_matrix_df_appended["d(d(x) /dt) /dt"])

In [334]:
dict(zip(alg_lasso.feature_names_in_, alg_lasso.coef_))

In [185]:
plt.plot(dummy_der["d(theta) /dt"], data_matrix_df_list[0]["omega"][:999])

In [184]:
(dummy_der["d(theta) /dt"] - data_matrix_df_list[0]["omega"][:999]).hist()

In [182]:
data_matrix_df_list[0]["u"][:999].hist()

In [31]:
pend_data.columns

In [37]:
pend_data = pd.read_csv("pendulum_data.txt")

pend_data.columns = pend_data.iloc[0]

pend_data = pend_data[1:]

pend_data = pend_data[list(pend_data.columns)[:-1]]

pend_data


In [47]:
pend_data["t"] = pend_data["t"].apply(lambda x: float(x))
pend_data["x"] = pend_data["x"].apply(lambda x: float(x))
pend_data["y"] = pend_data["y"].apply(lambda x: float(x))


In [53]:
pend_data["theta"] = np.arctan(-pend_data["y"]/pend_data["x"])

In [66]:
import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt

def deriv(t, y):
    """ODEs for Robertson's chemical reaction system."""
    x, y, z = y
    xdot = -0.04 * x + 1.e4 * y * z
    ydot = 0.04 * x - 1.e4 * y * z - 3.e7 * y**2
    zdot = 3.e7 * y**2
    return xdot, ydot, zdot

# Initial and final times.
t0, tf = 0, 500
t_eval = np.linspace(t0, tf, 1000)
# Initial conditions: [X] = 1; [Y] = [Z] = 0.
y0 = 1, 0, 0
# Solve, using a method resilient to stiff ODEs.
soln = solve_ivp(deriv, (t0, tf), y0, method='Radau', t_eval=t_eval)
print(soln.nfev, 'evaluations required.')

# Plot the concentrations as a function of time. Scale [Y] by 10**YFAC
# so its variation is visible on the same axis used for [X] and [Z].
YFAC = 4
plt.plot(soln.t, soln.y[0], label='[X]')
plt.plot(soln.t, 10**YFAC*soln.y[1], label=r'$10^{}\times$[Y]'.format(YFAC))
plt.plot(soln.t, soln.y[2], label='[Z]')
plt.xlabel('time /s')
plt.ylabel('concentration /arb. units')
plt.legend()
plt.show()

In [70]:
dat_mat = pd.DataFrame(soln.y.T, columns=["x","y","z"])

In [68]:
dat_mat["x"] + dat_mat["y"] + dat_mat["z"]

In [85]:
#Use lasso model by default
algebraic_model_1 = AlgModelFinder(model_id='lasso', alpha=0.5, fit_intercept=True)
algebraic_model_1.fit(dat_mat, scale_columns= True)


algebraic_model_1.best_models(5)

In [79]:
from dae_finder import AlgModelFinder





from dae_finder import sequentialThLin, AlgModelFinder
seq_th_model = sequentialThLin(fit_intercept=True, coef_threshold= 0.1)
algebraic_model_th = AlgModelFinder(custom_model=True, custom_model_ob= seq_th_model)


algebraic_model_th.fit(dat_mat, scale_columns= True)


algebraic_model_th.best_models(5)

In [81]:
algebraic_model_th.fit(dat_mat, scale_columns= False)


algebraic_model_th.best_models(5)

In [80]:
dat_mat.describe()